In [1]:
# Multilayer Perceptron (MLP) for multi-class softmax classification:
# modified from 
# https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam

import random

C:\Users\ortci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

n_samples = 1000000
n_partitions = 10

# Generate tasks and partitions
partition_data = np.random.random((n_samples, n_partitions)) # partition data generation

data = np.zeros((n_samples, n_partitions))                   # initialize input layer
labels = np.zeros((n_samples, n_partitions))                 # initialize outputs layer for training 
fit_data = np.zeros((n_samples, n_partitions))

task_data = np.zeros((n_samples, 1))                         # initialize task list

# fails to account a 'perfect' fit where fit == 0
for i in range (0, n_samples):
    
    partitions = partition_data[i]
    task = random.uniform(0, partitions.max())
    task_data[i] = task
    
    best_partition = -1
    best_fit = 999999999
    
    for j in range (0, n_partitions):
        current_fit = partitions[j] - task
        data[i,j] = current_fit
        if current_fit > 0:
            fit_data[i][j] = 1
            
            if current_fit < best_fit:
                best_fit = current_fit
                best_partition = j
        else:
            fit_data[i][j] = 0
            
    labels[i][best_partition] = 1
    

X = np.hstack((task_data,partition_data,fit_data))
y = labels

In [3]:
# split data between train and test set 
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
import tensorflow as tf
from keras.models import load_model

model = load_model('MLP_relu_softmax_21_inputs_20_outputs_v1.h5')

# Unfreeze all layers in first model
for layer in model.layers:
    layer.trainable = True

# Construct 2nd half model
n_nodes = 64
model.add(Dense(n_nodes, input_dim=2*n_partitions, activation='relu', name='layerA'))
model.add(Dense(n_nodes, activation='relu', name='layerB'))
model.add(Dense(n_partitions, activation='softmax', name='softmax'))

model.load_weights('simple_classifer_sign_inputs_wts.h5', by_name=True)

# optimizer options
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = tf.train.RMSPropOptimizer(0.008)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [5]:
# output current NN architecture to .png file
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.models import load_model
import pydot
import graphviz
from keras.utils import plot_model
plot_model(model, to_file='combined_model_config_21_inputs.png', show_shapes=True)

In [6]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.002),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [7]:
batchsize = 512

model.fit(X_train, y_train,
          epochs=40,
          batch_size=batchsize)
score = model.evaluate(X_test, y_test, batch_size=batchsize)
print(score)

Epoch 1/40
800000/800000 [==============================] - 108s 134us/step - loss: 0.6553 - categorical_accuracy: 0.7779
Epoch 2/40
800000/800000 [==============================] - 108s 135us/step - loss: 0.3719 - categorical_accuracy: 0.8436
Epoch 3/40
800000/800000 [==============================] - 108s 135us/step - loss: 0.3381 - categorical_accuracy: 0.8573
Epoch 4/40
800000/800000 [==============================] - 107s 134us/step - loss: 0.3156 - categorical_accuracy: 0.8664
Epoch 5/40
800000/800000 [==============================] - 108s 135us/step - loss: 0.3036 - categorical_accuracy: 0.8718
Epoch 6/40
800000/800000 [==============================] - 108s 135us/step - loss: 0.2927 - categorical_accuracy: 0.8765
Epoch 7/40
800000/800000 [==============================] - 112s 140us/step - loss: 0.2847 - categorical_accuracy: 0.8796
Epoch 8/40
800000/800000 [==============================] - 111s 139us/step - loss: 0.2782 - categorical_accuracy: 0.8827
Epoch 9/40
800000/800000

In [9]:
#### from keras.models import load_model

model.save('combined4b.h5')  # creates a HDF5 file 'my_model.h5'
model.save_weights('combined4b_weights.h5')

'''
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('MLP_Multiclass_softmax_10_inputs.h5')
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture
'''

C:\Users\ortci\Anaconda3\lib\site-packages\keras\engine\saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


"\ndel model  # deletes the existing model\n\n# returns a compiled model\n# identical to the previous one\nmodel = load_model('MLP_Multiclass_softmax_10_inputs.h5')\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture\n"

In [10]:
y_pred = model.predict(X_test, batch_size=batchsize)

In [11]:
# find cases for misclassification
misclass_set = []
for i in range(0,100):
    if (np.around(y_pred[i],0) != y_test[i]).any():
        misclass_set.append(i)
num_misclass = len(misclass_set)
print(num_misclass)

5


In [15]:
# show results of any specific case
test = misclass_set[2]

import pandas as pd

test_df = pd.DataFrame(np.vstack((X_test[test][1:n_partitions+1],
                                  np.full((1,10),X_test[test][0]),
                                  y_test[test][0:n_partitions],
                                  y_pred[test][0:n_partitions])))

test_df.index = ("Partition_AF",
                 "Task_Size",
                 "Actual_Best_Fit", # actual fit
                 "Pred_Best_Fit")   # actual fit
test_df

,0,1,2,3,4,5,6,7,8,9
Partition_AF,1.456530e-01,0.179568,5.513937e-02,0.797976,0.648353,0.069971,0.610844,0.974728,9.300118e-01,0.801615
Task_Size,7.684792e-01,0.768479,7.684792e-01,0.768479,0.768479,0.768479,0.768479,0.768479,7.684792e-01,0.768479
Actual_Best_Fit,0.000000e+00,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
Pred_Best_Fit,5.090148e-07,0.000008,4.937781e-07,0.481394,0.000001,0.000008,0.000007,0.000047,3.148277e-07,0.518534
